# SIIM-ISIC Melanoma Classification
## Identify melanoma in lesion images

In [2]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile, rmtree, move
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma'

In [7]:
len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/train'))

33126

In [5]:
len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/test'))

10982

In [19]:
source_benign_len = len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/source/benign'))
source_benign_len

32542

In [20]:
source_melignant_len = len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/source/melignant'))
source_melignant_len

584

In [21]:
source_benign_len + source_melignant_len

33126

In [12]:
local_zip = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/siim-isic-melanoma-classification.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma')
zip_ref.close()

In [15]:
len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/jpeg/train'))

33126

In [16]:
len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/jpeg/test'))

10982

In [18]:
# Read the train.csv file in order to get the target labels

df = pd.read_csv('train.csv')

In [19]:
df

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0
33122,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0
33123,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0
33124,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0


In [25]:
# Indicate the initial images directory path
jpeg_train_dir_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/jpeg/train'

# Create new directories for train and validation data
source_dir_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/source'
source_benign_path = os.path.join(source_dir_path, 'benign')
source_melignant_path = os.path.join(source_dir_path, 'melignant')
data_dir_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data'
train_dir_path = os.path.join(data_dir_path, 'train')
train_benign_dir_path = os.path.join(train_dir_path, 'benign')
train_melignant_dir_path = os.path.join(train_dir_path, 'melignant')
validation_dir_path = os.path.join(data_dir_path, 'validation')
validation_benign_dir_path = os.path.join(validation_dir_path, 'benign')
validation_melignant_dir_path = os.path.join(validation_dir_path, 'melignant')

try:
# Create the directories
    os.mkdir(source_dir_path)
    os.mkdir(source_benign_path)
    os.mkdir(source_melignant_path)
    os.mkdir(data_dir_path)
    os.mkdir(train_dir_path)
    os.mkdir(train_benign_dir_path)
    os.mkdir(train_melignant_dir_path)
    os.mkdir(validation_dir_path)
    os.mkdir(validation_benign_dir_path)
    os.mkdir(validation_melignant_dir_path)
   
except OSError:
    pass

In [ ]:
# Function that moves the images to separate benign and melignant folders
def identify_images(jpeg_train_dir_path, source_benign_path, source_melignant_path):

    list_source = os.listdir(jpeg_train_dir_path)
    
    for item in list_source:
        item_no = item[:-4]
        if df[df['image_name'] == item_no]['target'].values == 0:
            move(os.path.join(jpeg_train_dir_path,item), os.path.join(source_benign_path,item))
        elif df[df['image_name'] == item_no]['target'].values == 1:
            move(os.path.join(jpeg_train_dir_path,item), os.path.join(source_melignant_path,item))

identify_images(jpeg_train_dir_path, source_benign_path, source_melignant_path)

In [24]:
def split_data(source, train, validation, split_size):

    list_source = os.listdir(source)
    randomized_list = random.sample(list_source, len(list_source))
    
    for i, item in enumerate(randomized_list):
        if i < int(split_size*len(randomized_list)):
            copyfile(os.path.join(source,item), os.path.join(train,item))
        else:
            copyfile(os.path.join(source,item), os.path.join(validation,item))

split_size = .8
split_data(source_benign_path, train_benign_dir_path, validation_benign_dir_path, split_size)
split_data(source_melignant_path, train_melignant_dir_path, validation_melignant_dir_path, split_size)

KeyboardInterrupt: 

In [ ]:
len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data/train/benign'))

In [46]:
len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data/train/melignant'))

467

In [49]:
rmtree('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data')

In [6]:
jpeg_train_dir_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/jpeg/train'

# Create new directories for train and validation data
source_dir_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/source'
source_benign_path = os.path.join(source_dir_path, 'benign')
source_melignant_path = os.path.join(source_dir_path, 'melignant')
data_dir_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data'
train_dir_path = os.path.join(data_dir_path, 'train')
train_benign_dir_path = os.path.join(train_dir_path, 'benign')
train_melignant_dir_path = os.path.join(train_dir_path, 'melignant')
validation_dir_path = os.path.join(data_dir_path, 'validation')
validation_benign_dir_path = os.path.join(validation_dir_path, 'benign')
validation_melignant_dir_path = os.path.join(validation_dir_path, 'melignant')

In [7]:
def split_data(source, train, validation, split_size):

    list_source = os.listdir(source)
    randomized_list = random.sample(list_source, len(list_source))
    
    for i, item in enumerate(randomized_list):
        if i < int(split_size*len(randomized_list)):
            copyfile(os.path.join(source,item), os.path.join(train,item))
        else:
            copyfile(os.path.join(source,item), os.path.join(validation,item))

split_size = .8
split_data(source_benign_path, train_benign_dir_path, validation_benign_dir_path, split_size)
split_data(source_melignant_path, train_melignant_dir_path, validation_melignant_dir_path, split_size)

In [4]:
os.mkdir(data_dir_path)
os.mkdir(train_dir_path)
os.mkdir(train_benign_dir_path)
os.mkdir(train_melignant_dir_path)
os.mkdir(validation_dir_path)
os.mkdir(validation_benign_dir_path)
os.mkdir(validation_melignant_dir_path)